In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [6]:
tf.__version__

'2.11.0'

# import data

In [8]:
d=pd.read_csv(r'F:\ML\ML_part 1_2\Datasets\Churn_Modelling.csv');d.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
x=d.iloc[:,3:-1].values
y=d.iloc[:,-1].values

In [12]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [13]:
print(y)

[1 0 1 ... 1 1 0]


# Encoding

In [20]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,2]=le.fit_transform(x[:,2])
print(x)

[[1.0 0.0 0 ... 1 1 101348.88]
 [0.0 0.0 1 ... 0 1 112542.58]
 [1.0 0.0 0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0 ... 0 1 42085.58]
 [0.0 1.0 0 ... 1 0 92888.52]
 [1.0 0.0 0 ... 1 0 38190.78]]


In [22]:
# one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))

In [19]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Spillit test and train

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

# feature scaling

In [24]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

# Build ANN

In [25]:
ann=tf.keras.models.Sequential()

In [30]:
# add first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [31]:
# add second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [32]:
# add output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# train ANN

In [33]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [34]:
# taing ANN on training set
ann.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5630 - accuracy: 0.7933
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4965 - accuracy: 0.7933
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4826 - accuracy: 0.7933
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4538 - accuracy: 0.7933
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4367 - accuracy: 0.7933
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4312 - accuracy: 0.7933
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4283 - accuracy: 0.7933
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4259 - accuracy: 0.7940
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4243 - accuracy: 0.8034
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4223 - accura

# predict by ANN

Example:
geography france
credit 600
gender male
age 40
tenue 3 yeras
balance 60000
number of product 2
customer credit card yes
salary 50000

In [51]:
print(ann.predict(sc.transform([[1,0,0,1,608,0,41,1,83807,1,0,1,112542]]))>0.7)

1/1 [==============================] - 0s 18ms/step
[[False]]


In [56]:
y_pred=ann.predict(x_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 922us/step
[[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 0]]


In [57]:
# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1551   66]
 [ 254  129]]


0.84